# Very Advanced Problems on Defining Classes and Python's Object Model

These exercises go beyond basic class definitions and intentionally explore advanced aspects of:

- Custom attribute management (`__getattr__`, `__getattribute__`, `__setattr__`)
- Descriptors (`__get__`, `__set__`, `__set_name__`)
- Metaclasses and `type`
- `__new__` vs `__init__`
- `__init_subclass__` and class-level validation
- Multiple inheritance and MRO
- Class factories and dynamic class creation

Each problem is followed by a detailed solution implemented in code.

---
## Problem 1 — Implementing a Typed Descriptor

Create a descriptor `Typed` that:

1. Accepts a Python type (like `int`, `str`) in its constructor.
2. Uses `__set_name__(self, owner, name)` to remember the attribute name.
3. On assignment, checks that the value is an instance of the given type; otherwise raises `TypeError`.
4. Stores the data in the instance `__dict__` under the attribute name.

Then define a class `Person` that uses this descriptor:

```python
class Person:
    name = Typed(str)
    age = Typed(int)
```

Demonstrate that:
- `Person('Alice', 30)` works if you wire up `__init__` correctly.
- Assigning `p.age = 'old'` raises `TypeError`.

This problem tests your understanding of the descriptor protocol and attribute storage.

In [1]:
# === SOLUTION: Problem 1 ===
class Typed:
    def __init__(self, expected_type):
        self.expected_type = expected_type
        self._name = None

    def __set_name__(self, owner, name):
        self._name = name

    def __get__(self, instance, owner):
        if instance is None:
            return self  # access via class
        return instance.__dict__.get(self._name)

    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError(
                f"Expected {self.expected_type.__name__} for attribute '{self._name}', got {type(value).__name__}"
            )
        instance.__dict__[self._name] = value


class Person:
    name = Typed(str)
    age = Typed(int)

    def __init__(self, name, age):
        self.name = name
        self.age = age

    def __repr__(self):
        return f"Person(name={self.name!r}, age={self.age!r})"


p = Person("Alice", 30)
print(p)

try:
    p.age = "old"  # should raise TypeError
except TypeError as e:
    print("Error:", e)

Person(name='Alice', age=30)
Error: Expected int for attribute 'age', got str


---
## Problem 2 — Metaclass Enforcing Class Docstrings

Write a metaclass `DocEnforcer` that:

1. On class creation, checks whether the class has a non-empty docstring.
2. If not, raises a `TypeError` with a helpful message.

Use it like this:

```python
class Documented(metaclass=DocEnforcer):
    """I have a docstring."""
    pass

class Undocumented(metaclass=DocEnforcer):
    pass
```

The second class definition should fail at class creation time (before any instances exist).

This tests your ability to intercept and validate class definitions.

In [2]:
# === SOLUTION: Problem 2 ===
class DocEnforcer(type):
    def __new__(mcls, name, bases, namespace, **kwargs):
        cls = super().__new__(mcls, name, bases, namespace, **kwargs)
        doc = cls.__doc__
        if not doc or not doc.strip():
            raise TypeError(f"Class {name!r} must have a non-empty docstring")
        return cls


class Documented(metaclass=DocEnforcer):
    """I have a docstring."""
    pass

print("Documented created successfully:", Documented)

try:
    class Undocumented(metaclass=DocEnforcer):
        pass
except TypeError as e:
    print("Error while defining Undocumented:", e)

Documented created successfully: <class '__main__.Documented'>
Error while defining Undocumented: Class 'Undocumented' must have a non-empty docstring


---
## Problem 3 — `__new__` vs `__init__` and Instance Caching (Flyweight)

Implement a class `Symbol` that behaves like an interned string:

1. Internally maintain a cache (e.g. a dictionary at the class level) mapping string values to instances.
2. Override `__new__` so that calling `Symbol('x')` twice returns **the same instance** (identity equality with `is`).
3. The underlying value should be stored in an attribute `value`.
4. Demonstrate that `Symbol('x') is Symbol('x')` is `True`, but `Symbol('x') is Symbol('y')` is `False`.

This tests your understanding of `__new__` and controlling object identity.

In [3]:
# === SOLUTION: Problem 3 ===
class Symbol:
    _cache = {}

    def __new__(cls, value):
        if value in cls._cache:
            return cls._cache[value]
        self = super().__new__(cls)
        cls._cache[value] = self
        return self

    def __init__(self, value):
        # Note: __init__ may be called multiple times for the same instance.
        # We must ensure this is idempotent.
        self.value = value

    def __repr__(self):
        return f"Symbol({self.value!r})"


s1 = Symbol("x")
s2 = Symbol("x")
s3 = Symbol("y")

print("s1 is s2:", s1 is s2)
print("s1 is s3:", s1 is s3)
print("s1, s2, s3:", s1, s2, s3)

s1 is s2: True
s1 is s3: False
s1, s2, s3: Symbol('x') Symbol('x') Symbol('y')


---
## Problem 4 — Using `__init_subclass__` to Enforce Subclass Contracts

Create a base class `PluginBase` that:

1. Implements `__init_subclass__`.
2. Requires that every subclass defines a **class attribute** `name` (non-empty string).
3. Requires that every subclass implements a method `run(self)`.

If a subclass violates these rules, `__init_subclass__` should raise a `TypeError`.

Then:

```python
class ValidPlugin(PluginBase):
    name = 'valid'
    def run(self):
        return 'running'

class InvalidPlugin(PluginBase):
    def run(self):
        return 'oops'
```

The second class definition should fail because it does not define `name`.

This tests your ability to validate subclasses at definition time.

In [4]:
# === SOLUTION: Problem 4 ===
class PluginBase:
    def __init_subclass__(cls, **kwargs):
        super().__init_subclass__(**kwargs)
        # Check for 'name'
        name = getattr(cls, 'name', None)
        if not isinstance(name, str) or not name:
            raise TypeError(f"Subclass {cls.__name__!r} must define a non-empty class attribute 'name'")

        # Check for 'run'
        if 'run' not in cls.__dict__:
            raise TypeError(f"Subclass {cls.__name__!r} must implement a 'run(self)' method")


class ValidPlugin(PluginBase):
    name = 'valid'
    def run(self):
        return 'running'

print("ValidPlugin created:", ValidPlugin, "run() ->", ValidPlugin().run())

try:
    class InvalidPlugin(PluginBase):
        def run(self):
            return 'oops'
except TypeError as e:
    print("Error while defining InvalidPlugin:", e)

ValidPlugin created: <class '__main__.ValidPlugin'> run() -> running
Error while defining InvalidPlugin: Subclass 'InvalidPlugin' must define a non-empty class attribute 'name'


---
## Problem 5 — Logging Attribute Access with `__getattribute__`

Implement a class `LoggedAttributes` that:

1. Overrides `__getattribute__` to print a message every time any attribute is accessed:
   - The message should include the attribute name.
2. Overrides `__setattr__` to print a message every time an attribute is set.

Then derive a class `Account(LoggedAttributes)` with attributes `balance` and `owner` (set in `__init__`).

Demonstrate that accessing and modifying these attributes prints logs.

Be careful to use `super().__getattribute__` and `super().__setattr__` to avoid infinite recursion.

This tests deep understanding of attribute lookup internals.

In [5]:
# === SOLUTION: Problem 5 ===
class LoggedAttributes:
    def __getattribute__(self, name):
        print(f"[GET] {name!r} on {type(self).__name__}")
        return super().__getattribute__(name)

    def __setattr__(self, name, value):
        print(f"[SET] {name!r} = {value!r} on {type(self).__name__}")
        super().__setattr__(name, value)


class Account(LoggedAttributes):
    def __init__(self, owner, balance):
        self.owner = owner
        self.balance = balance

    def deposit(self, amount):
        self.balance += amount


acc = Account("Alice", 100)
print("Owner:", acc.owner)
acc.deposit(50)
print("Balance:", acc.balance)

[SET] 'owner' = 'Alice' on Account
[SET] 'balance' = 100 on Account
[GET] 'owner' on Account
Owner: Alice
[GET] 'deposit' on Account
[GET] 'balance' on Account
[SET] 'balance' = 150 on Account
[GET] 'balance' on Account
Balance: 150


---
## Problem 6 — Multiple Inheritance, `super()`, and MRO

Consider a diamond-shaped inheritance hierarchy:

```text
      Base
     /    \
   Left  Right
     \    /
     Final
```

1. Implement `Base` with a method `whoami(self)` that returns the string `"Base"`.
2. Implement `Left(Base)` overriding `whoami` to prepend `"Left -> "` to `super().whoami()`.
3. Implement `Right(Base)` overriding `whoami` to prepend `"Right -> "` to `super().whoami()`.
4. Implement `Final(Left, Right)` that does not override `whoami` but inherits it.

Demonstrate:
- The MRO of `Final`.
- The result of `Final().whoami()`.

Explain via code why using `super()` correctly walks the MRO only once.

This tests your understanding of method resolution order in complex inheritance graphs.

In [6]:
# === SOLUTION: Problem 6 ===
class Base:
    def whoami(self):
        return "Base"


class Left(Base):
    def whoami(self):
        return "Left -> " + super().whoami()


class Right(Base):
    def whoami(self):
        return "Right -> " + super().whoami()


class Final(Left, Right):
    pass


f = Final()
print("MRO of Final:")
for cls in Final.__mro__:
    print(" ", cls.__name__)

print("Final().whoami():", f.whoami())

MRO of Final:
  Final
  Left
  Right
  Base
  object
Final().whoami(): Left -> Right -> Base


---
## Problem 7 — Class Factory with `type` and Custom `__repr__`

Write a function `record_class(name, field_names)` that:

1. Dynamically creates and returns a new class using `type`.
2. `field_names` is an iterable of strings representing attribute names.
3. The generated class has:
   - An `__init__` that accepts positional arguments corresponding to `field_names` and stores them on `self`.
   - A `__repr__` that prints the class name and field values, e.g. `Point(x=1, y=2)`.

Example usage:

```python
Point = record_class('Point', ['x', 'y'])
p = Point(1, 2)
print(p)   # should show Point(x=1, y=2)
```

This tests your ability to synthesize classes programmatically using the `type` constructor.

In [7]:
# === SOLUTION: Problem 7 ===
def record_class(name, field_names):
    field_names = tuple(field_names)

    def __init__(self, *args, **kwargs):
        if len(args) + len(kwargs) != len(field_names):
            raise TypeError(
                f"Expected {len(field_names)} arguments, got {len(args) + len(kwargs)}"
            )
        # Assign positional args
        for fname, value in zip(field_names, args):
            setattr(self, fname, value)
        # Assign keyword args
        for fname in field_names[len(args):]:
            if fname not in kwargs:
                raise TypeError(f"Missing value for field {fname!r}")
            setattr(self, fname, kwargs[fname])

    def __repr__(self):
        parts = []
        for fname in field_names:
            parts.append(f"{fname}={getattr(self, fname)!r}")
        inner = ", ".join(parts)
        return f"{name}({inner})"

    namespace = {
        '__slots__': field_names,  # optional optimization
        '__init__': __init__,
        '__repr__': __repr__
    }

    return type(name, (object,), namespace)


Point = record_class('Point', ['x', 'y'])
p = Point(1, 2)
print(p)
print("Slots:", Point.__slots__)

Point(x=1, y=2)
Slots: ('x', 'y')


---
## Problem 8 — Fallback Attribute Handling with `__getattr__`

Implement a class `Config` that behaves like a lazy configuration object:

1. Its constructor takes a dictionary and stores it internally (e.g. in `_data`).
2. If an attribute is not found via the normal mechanism, `__getattr__` should be called and:
   - Look up the attribute name as a key in `_data`.
   - If found, return the value.
   - Otherwise, raise `AttributeError` with a helpful message.

Example usage:

```python
cfg = Config({'host': 'localhost', 'port': 8080})
cfg.host  # 'localhost'
cfg.port  # 8080
cfg.debug # should raise AttributeError
```

This tests your understanding of the difference between `__getattribute__` and `__getattr__` and how to use the latter as a fallback.

In [8]:
# === SOLUTION: Problem 8 ===
class Config:
    def __init__(self, data):
        self._data = dict(data)

    def __getattr__(self, name):
        # Called only if normal attribute lookup fails
        if name in self._data:
            return self._data[name]
        raise AttributeError(f"Config has no attribute or key {name!r}")

    def __repr__(self):
        return f"Config({self._data!r})"


cfg = Config({'host': 'localhost', 'port': 8080})
print("host:", cfg.host)
print("port:", cfg.port)
try:
    print(cfg.debug)
except AttributeError as e:
    print("Error:", e)

host: localhost
port: 8080
Error: Config has no attribute or key 'debug'


---
## Problem 9 — Combining Descriptors with Metaclasses for a Mini ORM

Design a minimalistic ORM-like system where:

1. `Field` is a descriptor base class representing a mapped column.
   - It stores the field name via `__set_name__`.
2. Concrete subclasses like `IntegerField` and `StringField` enforce simple typing rules.
3. A metaclass `ModelMeta` collects all `Field` instances defined on the class and stores them in a class-level `__fields__` dictionary.
4. A base class `Model(metaclass=ModelMeta)` represents a database row; its `__init__` accepts field values via keyword arguments and assigns them using the descriptors.

Example usage:

```python
class User(Model):
    id = IntegerField()
    name = StringField()

u = User(id=1, name='Alice')
print(User.__fields__)  # should map 'id' and 'name' to the corresponding Field objects
```

This tests combining multiple advanced mechanisms (descriptors, `__set_name__`, and metaclasses) into a coherent design.

In [9]:
# === SOLUTION: Problem 9 ===
class Field:
    def __init__(self):
        self.name = None

    def __set_name__(self, owner, name):
        self.name = name

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.__dict__.get(self.name)

    def __set__(self, instance, value):
        # default: no validation
        instance.__dict__[self.name] = value


class IntegerField(Field):
    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise TypeError(f"Field {self.name!r} expects int, got {type(value).__name__}")
        super().__set__(instance, value)


class StringField(Field):
    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise TypeError(f"Field {self.name!r} expects str, got {type(value).__name__}")
        super().__set__(instance, value)


class ModelMeta(type):
    def __new__(mcls, name, bases, namespace, **kwargs):
        cls = super().__new__(mcls, name, bases, namespace, **kwargs)
        fields = {}
        # Inherit fields from base classes
        for base in reversed(cls.__mro__[1:]):
            if hasattr(base, '__fields__'):
                fields.update(base.__fields__)
        # Add fields defined on this class
        for attr_name, attr_value in namespace.items():
            if isinstance(attr_value, Field):
                fields[attr_name] = attr_value
        cls.__fields__ = fields
        return cls


class Model(metaclass=ModelMeta):
    def __init__(self, **kwargs):
        for name, field in self.__fields__.items():
            if name in kwargs:
                setattr(self, name, kwargs[name])
            else:
                # optional: allow missing values; could default to None
                setattr(self, name, None)

    def __repr__(self):
        pairs = []
        for name in self.__fields__:
            pairs.append(f"{name}={getattr(self, name)!r}")
        inner = ", ".join(pairs)
        return f"{type(self).__name__}({inner})"


class User(Model):
    id = IntegerField()
    name = StringField()


u = User(id=1, name='Alice')
print("User instance:", u)
print("User.__fields__:")
for name, field in User.__fields__.items():
    print(" ", name, "->", field)

try:
    u.id = 'not an int'
except TypeError as e:
    print("Type error when setting id:", e)

User instance: User(id=1, name='Alice')
User.__fields__:
  id -> <__main__.IntegerField object at 0x000001B523798D70>
  name -> <__main__.StringField object at 0x000001B523798EC0>
Type error when setting id: Field 'id' expects int, got str
